# Pima Indians Diabetes dataset for Feature Selection
---
Si vuole utilizzare il dataset PID, che è costituito da diverse variabili predittive mediche (indipendenti) e una variabile obiettivo (dipendente), Outcome. Le variabili indipendenti includono svariate caratteristiche, come ad esempio il numero di gravidanze che la paziente ha avuto, il loro BMI, il livello di insulina, l'età e così via.

Cominciamo con l'ìmportare le librerie di interesse (numpy per le operazioni algebriche e pandas per il data processing e per l'I/O del dataset):

In [ ]:
import numpy as np 
import pandas as pd 
import sklearn as sk
import sklearn.feature_selection
import os

#Set the log level in order to hide the unuseful warnings
import logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1" 

pima_ds = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")

Possiamo dare uno sguardo al dataset appena caricato invocando la funzione *head()*: questo ci permette di capire com'è fatto quel dataset.

In [ ]:
pima_ds.head()

Ad esempio, il campo **Outcome** stabilisce se alla persona è stato diagnosticato il diabete (1) oppure no (0).

Alcuni valori sono lasciati a 0: significa che per quei valori non è stata disposta una misurazione. 

Ora è il momento di convertire il dataset in un array, separando i dati in variabili differenti in modo da separare le caratteristiche di ognuna:

In [ ]:
array = pima_ds.values
X = array[:,0:8]
Y = array[:,8]

## Test del Chi Quadrato

Innanzitutto, verrà implementato il test del Chi Quadrato per le caratteristiche non negative: questo per selezionare le 4 migliori caratteristiche dal set di dati.

In [ ]:
test = sk.feature_selection.SelectKBest(score_func=sk.feature_selection.chi2, k=4)
fit = test.fit(X, Y)

# Summarize scores
np.set_printoptions(precision=3)
print(fit.scores_)

features = fit.transform(X)
# Summarize selected features
print(features[0:5,:])

**Osservazione:** i pesi per ogni attributo e i 4 attributi scelti (quelli con i punteggi più alti) sono: *Glucose*, *Insuline*, *BMI* ed *Age*. Questi punteggi sono un buon indicatore per determinare le migliori caratteristiche per l'addestramento di un ipotetico modello.

---


## Recursive Feature Elimination
La Recursive Feature Elimination (o RFE) funziona rimuovendo in modo ricorsivo gli attributi e costruendo un modello sugli attributi che rimangono alla fine della procedura.
RFE utilizza la precisione del modello per identificare gli attributi (e la combinazione di attributi) che contribuiscono maggiormente alla previsione dell'attributo target.

Verrà utilizzata la RFE con il classificatore *Logistic Regression* per trovare le 3 caratteristiche migliori. In questo caso, la scelta dell'algoritmo non impatta significativamente.

In [ ]:
# Feature extraction
model = sk.linear_model.LogisticRegression(max_iter=1000)
rfe = sk.feature_selection.RFE(model, n_features_to_select=3)
fit = rfe.fit(X, Y)
print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % (fit.support_))
print("Feature Ranking: %s" % (fit.ranking_))

**Osservazione:** la RFE, come da aspettative, ha selezionato i tre attributi migliori: *Pregnancies*, *BMI* e *DiabetesPedigreeFunction*. Queste caratteristiche sono marchiate **"True** nella matrice di supporto, con una scelta "1" nella matrice di classificazione. Questo, a sua volta, indica il peso di queste caratteristiche.

## Ridge Regression
La Ridge Regression è fondamentalmente una tecnica di regolarizzazione e, allo stesso tempo, anche una tecnica di selezione delle caratteristiche.

In [ ]:
ridge = sk.linear_model.Ridge(alpha=1.0)
ridge.fit(X,Y)

print(str(ridge.coef_))

I risultati ottenuti sono i coefficienti delle incognite $X_i$. Infatti il modello ottenuto applicando la regressione di Ridge può essere riscritto in questo modo:
$$0.021 X_0 + 0.006 X_1 + -0.002 X_2 + 0 X_3 + 0 X_4 + 0.013 X_5 + 0.145 X_6 + 0.003 X_7$$

## Conclusioni
I metodi che sono stati implementati aiutano a comprendere le caratteristiche di un particolare set di dati in modo completo. La *Feature Selection* è essenzialmente una parte del *preprocessing* dei dati che è considerata la parte più dispendiosa in termini di tempo, in qualsiasi pipeline dei *task* di machine learning.
Le tecniche esposte servono, quindi, ad affrontare in modo più sistematico e chiaro le problematiche legate ad alcune task di apprendimento automatico: esse, infatti, aiutano l'interpretazione delle caratteristiche, presentandole in maniera più accurata ed evidenziando quelle più rilevanti. 